In [1]:
import sys
sys.path.append('../misc')

from MoviaBusDataset import MoviaBusDataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.init import kaiming_normal
from torch.utils.data import DataLoader
import numpy as np

In [2]:
prev_timesteps = 2
prediction_steps = 1

train = MoviaBusDataset('../data/train', interpolation=True, prev_timesteps=prev_timesteps, prediction_steps=prediction_steps)
test = MoviaBusDataset('../data/test', interpolation=True, prev_timesteps=prev_timesteps, prediction_steps=prediction_steps)

train.normalize()
test.normalize(train.mean, train.std)

train_DL = DataLoader(train, batch_size=50, shuffle=False)
test_DL = DataLoader(test, batch_size=50, shuffle=False)

In [3]:
num_input = prev_timesteps + 1

class FNN(nn.Module):
    def __init__(self, num_hidden1):
        super(FNN, self).__init__()
    
        self.l1 = nn.Linear(num_input, num_hidden1)
        self.l2 = nn.Linear(num_hidden1, num_hidden1)
        self.l3 = nn.Linear(num_hidden1, 1)
        self.activation = nn.ReLU()
        
    def forward(self,x):
        
        x = self.activation(self.l1(x))
        x = self.activation(self.l2(x))

        x = self.l3(x)
        return x

net = FNN(10)

optimizer = optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.L1Loss()

In [4]:

for epoch in range(100):
    cur_loss = []
    for i , batch in enumerate(train_DL):
        output = net(batch['data'].transpose(2,1)).squeeze()[:,:-1]
        target = batch['target']
        
        #Disgard missing data points, and speeds of 0, since this isn't usefull for us
        output = output[(torch.isnan(target)==0) & (target!=0)]
        target = target[(torch.isnan(target)==0) & (target!=0)]
        optimizer.zero_grad()

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        cur_loss.append(loss.item())
    
    print("epoch = %2i  loss = %0.3f" %(epoch, np.mean(cur_loss)))

epoch =  0  loss = 0.757
epoch =  1  loss = 0.757
epoch =  2  loss = 0.757
epoch =  3  loss = 0.757
epoch =  4  loss = 0.757
epoch =  5  loss = 0.757
epoch =  6  loss = 0.757
epoch =  7  loss = 0.757
epoch =  8  loss = 0.757
epoch =  9  loss = 0.757
epoch = 10  loss = 0.757
epoch = 11  loss = 0.757
epoch = 12  loss = 0.757
epoch = 13  loss = 0.757
epoch = 14  loss = 0.757
epoch = 15  loss = 0.757
epoch = 16  loss = 0.756
epoch = 17  loss = 0.756
epoch = 18  loss = 0.756
epoch = 19  loss = 0.756
epoch = 20  loss = 0.756
epoch = 21  loss = 0.756
epoch = 22  loss = 0.756
epoch = 23  loss = 0.756
epoch = 24  loss = 0.756
epoch = 25  loss = 0.756
epoch = 26  loss = 0.756
epoch = 27  loss = 0.756
epoch = 28  loss = 0.756
epoch = 29  loss = 0.756
epoch = 30  loss = 0.756
epoch = 31  loss = 0.756
epoch = 32  loss = 0.756
epoch = 33  loss = 0.756
epoch = 34  loss = 0.756
epoch = 35  loss = 0.756
epoch = 36  loss = 0.756
epoch = 37  loss = 0.756
epoch = 38  loss = 0.756
epoch = 39  loss = 0.756


In [5]:
for i, batch in enumerate(test_DL):
    output = net(batch['data'].transpose(2,1)).squeeze()[:,:-1]
    target = batch['target']
    
    output = (output + torch.tensor(train.mean)[:-1])*torch.tensor(train.std)[:-1]
    target = (target + torch.tensor(train.mean)[:-1])*torch.tensor(train.std)[:-1]
    
    output = output[torch.isnan(target)==0]
    target = target[torch.isnan(target)==0]
    
    loss = criterion(output,target)
    print(loss.item())

1.8180862665176392
1.5114802122116089
1.7001041173934937
1.6605123281478882
1.731291651725769
1.6738601922988892
1.8250737190246582
1.7471632957458496
